# Python 语法糖 #

# list推导式（适用于pyspark df.column, Pandas df.series）

In [1]:
squares = [x**2 for x in range(10)]
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [2]:
square_dict = {x: x**2 for x in range(5)}
print(square_dict)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}


In [3]:
square_set = {x**2 for x in [1, 2, 3, 2]}
print(square_set)

{1, 4, 9}


# 生成器表达式:类似于列表推导，但提供了一个生成器而不是列表

In [4]:
sum_of_squares = sum(x**2 for x in range(10))
print(sum_of_squares)

285


In [5]:
a=[1,2,3]
b=[1,2,3]
print(id(a))
print(id(b))

2484951139392
2484951134464


In [11]:
a==b

True

In [12]:
a is b

False

In [6]:
a=b
print(id(a))
print(id(b))

1841920695872
1841920695872


In [8]:
b=[1,2,3,4]
print(id(a))
print(id(b))

1841920695872
1841933925632


# yield和return的区别，每个函数只return一次，但是可以yield很多次，yield返回的是一个生成的对象，可以迭代多次

In [6]:
def hello():
    print('Step 1')
    yield 1
    print('Step 2')
    yield 2
    print('Step 3')
    yield 3
# 调用不能直接调用Hello(), 直接调用该函数就只打印一个Step 1 就结束了

g=hello()
for i in g:
    i


Step 1
Step 2
Step 3


In [7]:
def squares(count:int):
    for i in range(count):
        yield i**2

g=squares(5)
for i in g:
    print(i)

0
1
4
9
16


In [8]:
def spark():
    spark='a'
    # return spark 返回'a'， yield spark 返回spark object,本质上都是spark对象
    yield spark

g=spark()
g

<generator object spark at 0x0000024292833E80>

In [9]:
numbers = [1, 2, 3, 4]
result = map(lambda x: x * 2, numbers)
#原来map()函数会返回列表，在Python3中map()函数返回一个map对象可以使用list(map)来查看每个元素
print(list(result))

[2, 4, 6, 8]


In [10]:
numbers = [1, 2, 3, 4, 5, 6]
result = filter(lambda x: x % 2 == 0, numbers)
print(list(result))

[2, 4, 6]


In [11]:
from functools import reduce
numbers = [1, 2, 3, 4]
result = reduce(lambda x, y: x * y, numbers)
print(result)

24


In [5]:
from dataclasses import dataclass

@dataclass
class Student:
    name:str
    age:int
    
#对象创建后不能改变对象的属性
Tom=Student('Tom',23)
print(Tom.age)

23


# 涉及精度取小数位数的一律使用decimal，不要使用float,float类型会导致四舍五入是产生精度错误

In [13]:
import logging
        
logging.basicConfig(filename='log/log.txt')
log=logging.getLogger('mylogger')
log.setLevel(logging.DEBUG)
log.info("This is info logger")

class Student:
    def __init__(self,name,age):
        self.name=name
        self.age=age

       

tom=Student('Tom',18)
log.info(tom.name)
log.info(tom.age)

In [14]:
import re

# query = """select site_key, INITIAL_channel, INITIAL_channel as ATLANTIS_auto_channel from default.ma_eda_features """
query = """HFS,select site_key, INITIAL_channel, case when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month <= 1.02 then "HFS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience <= 0.00 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience <= 0.01 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience > 0.01 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant > 1.50 and avg_num_sub_brand_month > 1.38 then "MM" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month > 1.02 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant > 6.50 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month > 5.83 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant <= 6.50 then "WS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience > 0.10 then "WS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience > 0.00 then "WS" else INITIAL_channel end as ATLANTIS_auto_channel from default.ma_eda_features """
case_statements = re.findall(r'when(.*?)then "(.*?)"', query, re.DOTALL)

results = []
for conditions, result in case_statements:
    conditions = conditions.strip()
    conditions = re.split(r'\s+and\s+', conditions)  # Split conditions on 'and'
    formatted = ['Channel1']  # Assume a default channel
    formatted.append(result)
    formatted.extend(conditions)
    results.append(','.join(formatted))

for line in results:
    print(line)

Channel1,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
Channel1,HFS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
Channel1,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
Channel1,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
Channel1,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38
Channel1,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month > 1.02
Channel1,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month > 0.80,avg_num_sub_brand_month <= 5.83,Feminine_Care_Variant > 6.50
Channel1,MM,avg_sales_value_week > 2112.08,Bo

In [15]:
decision_paths = {'WHOLESALER: HYPER/ SUPER/ HFS': ['']}
if all(not s for value_list in decision_paths.values() for s in value_list):
    print("Empty")

Empty


In [16]:
import re

# 原始的 SQL 查询
query = """HFS,select site_key, INITIAL_channel, case when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month <= 1.02 then "HFS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience <= 0.00 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience <= 0.01 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience > 0.01 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant > 1.50 and avg_num_sub_brand_month > 1.38 then "MM" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month > 1.02 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant > 6.50 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month > 5.83 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant <= 6.50 then "WS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience > 0.10 then "WS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience > 0.00 then "WS" else INITIAL_channel end as ATLANTIS_auto_channel from default.ma_eda_features"""

# 提取 case 语句中的条件和结果
case_parts = re.findall(r'when (.*?) then "(.*?)"', query)

# 获取原始的第一部分，也即第一个逗号前的内容
first_part = query.split(',')[0]

# 格式化输出
results = []
for condition, result in case_parts:
    conditions = condition.split(' and ')
    formatted_line = [first_part, result] + conditions
    results.append(','.join(formatted_line))

for line in results:
    print(line)

HFS,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
HFS,HFS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
HFS,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
HFS,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38
HFS,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month > 1.02
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month > 0.80,avg_num_sub_brand_month <= 5.83,Feminine_Care_Variant > 6.50
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month > 0.80,avg_num_s

In [17]:
import re

# 多个查询组成的列表
queries = [
    """HFS,select site_key, INITIAL_channel, case when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month <= 1.02 then "HFS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience <= 0.00 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience <= 0.01 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience > 0.01 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant > 1.50 and avg_num_sub_brand_month > 1.38 then "MM" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month > 1.02 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant > 6.50 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month > 5.83 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant <= 6.50 then "WS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience > 0.10 then "WS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience > 0.00 then "WS" else INITIAL_channel end as ATLANTIS_auto_channel from default.ma_eda_features""",
    """WS,select site_key, INITIAL_channel, case when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month <= 1.02 then "HFS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience <= 0.00 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience <= 0.01 then "HFS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant > 77.50 and Home_Care_salience > 0.01 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant > 1.50 and avg_num_sub_brand_month > 1.38 then "MM" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience <= 0.10 and Total_Variant <= 77.50 and Bottom_SKU_case_per_month > 1.02 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and avg_num_sub_brand_month <= 5.83 and Feminine_Care_Variant > 6.50 then "MM" when average_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and average_num_sub_brand_month > 5.83 then "MM" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month > 0.80 and average_num_sub_brand_month <= 5.83 and Feminine_Care_Variant <= 6.50 then "WS" when avg_sales_value_week <= 2112.08 and Fabric_Care_perf_cond_salience > 0.10 then "WS" when avg_sales_value_week > 2112.08 and Bottom_SKU_case_per_month <= 0.80 and Home_Care_Variant <= 1.50 and Hair_Care_perf_cond_salience > 0.00 then "WS" else INITIAL_channel end as ATLANTIS_auto_channel from default.ma_eda_features"""
]

formatted_result = []

for query in queries:
    first_part = query.split(',')[0]
    case_parts = re.findall(r'when\s+(.*?)\s+then\s+"(.*?)"', query)

    for condition, result in case_parts:
        conditions = condition.split(' and ')
        formatted_line = [first_part, result] + conditions
        formatted_result.append(','.join(formatted_line) + '\n')

final_output = ''.join(formatted_result)
print(final_output)

HFS,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
HFS,HFS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
HFS,HFS,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
HFS,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38
HFS,MM,avg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month > 1.02
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month > 0.80,avg_num_sub_brand_month <= 5.83,Feminine_Care_Variant > 6.50
HFS,MM,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month > 0.80,avg_num_s

In [18]:
import re

def format_expression(expression):
    pattern = r'([^\s]+%)\s*(>=|<=|>|<)\s*'
    
    def repl(match):
        first_part = '`{}`'.format(match.group(1))
        return '{} {} '.format(first_part, match.group(2))
    
    formatted_expression = re.sub(pattern, repl, expression)
    formatted_expression = re.sub(r'\s+and\s+', ' and ', formatted_expression)
    formatted_expression = re.sub(r'\s+or\s+', ' or ', formatted_expression)

    return formatted_expression


expression = "Home_Care_salience <= 0.00 and Feminine_Care_salience > 0.09 and Feminine_Care_Variant <= 6.50 and sales_value_monthly <= 763.81 and SKU_2_3_% > 0.00"
formatted_expression = format_expression(expression)
print(formatted_expression)


Home_Care_salience <= 0.00 and Feminine_Care_salience > 0.09 and Feminine_Care_Variant <= 6.50 and sales_value_monthly <= 763.81 and `SKU_2_3_%` > 0.00


In [20]:
import re

sql = """
        select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end as auto_channel
        from default.ma_eda_features
"""

case_part = re.findall(r"case(.*?)end as", sql, re.DOTALL)[0]

conditions = re.findall(r"when(.*?)then '(.*?)'", case_part, re.DOTALL)

output = []
formatted_conditions = []
for cond in condition.split(' and '):
    if '%' in re.split(r'[<>]', cond)[0]:
        formatted_conditions.append(cond.strip())
    else:
        formatted_conditions.append(cond.replace("`", "").strip())
    output_line = f",{result}," + ",".join(formatted_conditions)
    output.append(output_line)

for line in output:
    print(line)

,WS,avg_sales_value_week > 2112.08
,WS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80
,WS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50
,WS,avg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience > 0.00


In [21]:
import os
image_path="/dbfs/pyrogai_run/dev/2024/11/13/2024-11-13T07-33-51_1731483228.4545808/auto_channel/depth_2_tree.png"
file_name = os.path.basename(image_path)
print(file_name)

depth_2_tree.png


In [23]:
sql = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end as auto_channel
        from default.ma_eda_features
"""


# 提取 CASE 部分
case_part = re.findall(r"case(.*?)end\s*", sql, re.DOTALL)
if case_part:
    case_content = case_part[0]

# 通过'when' 和 'then' 解析所有条件和结果
conditions = re.findall(r"when(.*?)then \"(.*?)\"", case_content, re.DOTALL)

output = []
for condition, result in conditions:
    # 移除条件中不必要的单引号（如果操作符前没有%）
    formatted_conditions = []
    for cond in condition.split(' and '):
        if '%' in re.split(r'[<>]', cond)[0]:
            formatted_conditions.append(cond.strip())
        else:
            formatted_conditions.append(cond.replace("`", "").strip())
    output_line = f",{result}," + ",".join(formatted_conditions)
    output.append(output_line)

for line in output:
    print(line)

,HFS,avg_sales_value_week <= 2112.08,`Fabric_Care_perf_cond_rate%` <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
,HFS,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
,HFS,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
,MM,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
,MM,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38


In [24]:
import re

sql = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end
        from default.ma_eda_features
"""

# 尝试匹配 case 到 end，包含多种可能的结尾
case_part = re.findall(r"case(.*?)end\s*", sql, re.DOTALL)
if case_part:
    case_content = case_part[0]
    print("找到匹配：", case_content.strip())
else:
    print("未找到匹配项。")

找到匹配： when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"


In [25]:
import re

sql = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end
        from default.ma_eda_features
"""

case_parts = re.findall(r"case(.*?)end", sql, re.DOTALL)
if case_parts:
    case_content = case_parts[0]

    # get condition part between when and then
    conditions = re.findall(r"when(.*?)then \"(.*?)\"", case_content, re.DOTALL)

    output = []
    for condition, result in conditions:
        # remove `` when there is no % in the column name
        formatted_conditions = []
        for cond in condition.split(' and '):
            if '%' in re.split(r'[<>]', cond)[0]:
                formatted_conditions.append(cond.strip())
            else:
                formatted_conditions.append(cond.replace("`", "").strip())
        output_line = f"{result}," + ",".join(formatted_conditions)
        output.append(output_line)
    parsed_result = ("\n".join(output))
else:
    print("there is no case string")
    parsed_result = None
    
print(parsed_result)

HFS,avg_sales_value_week <= 2112.08,`Fabric_Care_perf_cond_rate%` <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
HFS,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
HFS,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
MM,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
MM,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38


In [26]:
import re 
def parse_default_stage_rules(query_string):
    # Get case part
    case_parts = re.findall(r"case(.*?)end", query_string, re.DOTALL)
    if case_parts:
        case_content = case_parts[0]

        # get condition part between when and then
        conditions = re.findall(r"when(.*?)then \"(.*?)\"", case_content, re.DOTALL)

        output = []
        for condition, result in conditions:
            # remove `` when there is no % in the column name
            formatted_conditions = []
            for cond in condition.split(' and '):
                if '%' in re.split(r'[<>]', cond)[0]:
                    formatted_conditions.append(cond.strip())
                else:
                    formatted_conditions.append(cond.replace("`", "").strip())
            output_line = f"{result}," + ",".join(formatted_conditions)
            output.append(output_line)
        parsed_result = ("\n".join(output))
    else:
        parsed_result = None
    return parsed_result

sql = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end
        from default.ma_eda_features
"""


parse_default_stage_rules(sql)

'HFS,avg_sales_value_week <= 2112.08,`Fabric_Care_perf_cond_rate%` <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02\nHFS,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00\nHFS,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01\nMM,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01\nMM,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38'

In [27]:
query_string = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end
        from default.ma_eda_features
"""
case_parts = re.findall(r"case(.*?)end", query_string, re.DOTALL)
for case_part in case_parts:
    conditions = re.findall(r"when(.*?)then \"(.*?)\"", case_part, re.DOTALL)
    output = []
    for condition, result in conditions:
        # remove `` when there is no % in the column name
        formatted_conditions = []
        for cond in condition.split(' and '):
            if '%' in re.split(r'[<>]', cond)[0]:
                formatted_conditions.append(cond.strip())
            else:
                formatted_conditions.append(cond.replace("`", "").strip())
        output_line = f"{result}," + ",".join(formatted_conditions) + "\n"
        output.append(output_line)
    parsed_result = ("".join(output))
print(parsed_result)

HFS,avg_sales_value_week <= 2112.08,`Fabric_Care_perf_cond_rate%` <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02
HFS,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00
HFS,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01
MM,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01
MM,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38



In [28]:
def parse_default_stage_rules(query_string):
    # Get case part
    case_parts = re.findall(r"case(.*?)end", query_string, re.DOTALL)
    for case_part in case_parts:
        conditions = re.findall(r"when(.*?)then \"(.*?)\"", case_part, re.DOTALL)
        output = []
        for condition, result in conditions:
            # remove `` when there is no % in the column name
            formatted_conditions = []
            for cond in condition.split(' and '):
                if '%' in re.split(r'[<>]', cond)[0]:
                    formatted_conditions.append(cond.strip())
                else:
                    formatted_conditions.append(cond.replace("`", "").strip())
            output_line = f"{result}," + ",".join(formatted_conditions) + "\n"
            output.append(output_line)
        return ("".join(output))

query_string = """
    select site_key, INITIAL_channel,
        case
        when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
        when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
        when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
        end
        from default.ma_eda_features
"""
parse_default_stage_rules(query_string)


'HFS,avg_sales_value_week <= 2112.08,`Fabric_Care_perf_cond_rate%` <= 0.10,Total_Variant <= 77.50,Bottom_SKU_case_per_month <= 1.02\nHFS,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant <= 1.50,Hair_Care_perf_cond_salience <= 0.00\nHFS,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience <= 0.01\nMM,vg_sales_value_week <= 2112.08,Fabric_Care_perf_cond_salience <= 0.10,Total_Variant > 77.50,Home_Care_salience > 0.01\nMM,vg_sales_value_week > 2112.08,Bottom_SKU_case_per_month <= 0.80,Home_Care_Variant > 1.50,avg_num_sub_brand_month > 1.38\n'

In [30]:
import re

query_string = """
select site_key, INITIAL_channel,
case
when `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"
when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"
when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"
when `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"
when `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"
end as auto_channel
from default.ma_eda_features
"""
case_parts = re.findall(r"case(.*?)end", query_string, re.DOTALL)
print(case_parts)

['\nwhen `avg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_rate%` <= 0.10 and `Total_Variant` <= 77.50 and `Bottom_SKU_case_per_month` <= 1.02 then "HFS"\nwhen `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` <= 1.50 and `Hair_Care_perf_cond_salience` <= 0.00 then "HFS"\nwhen `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` <= 0.01 then "HFS"\nwhen `vg_sales_value_week` <= 2112.08 and `Fabric_Care_perf_cond_salience` <= 0.10 and `Total_Variant` > 77.50 and `Home_Care_salience` > 0.01 then "MM"\nwhen `vg_sales_value_week` > 2112.08 and `Bottom_SKU_case_per_month` <= 0.80 and `Home_Care_Variant` > 1.50 and `avg_num_sub_brand_month` > 1.38 then "MM"\n']


In [31]:
def parse_default_stage_rules(query_string):
    # Get case part
    case_part = re.findall(r"case(.*?)end", query_string, re.DOTALL)[0]
    output = []
    result = []

    conditions = re.findall(r"when(.*?)then '(.*?)'", case_part, re.DOTALL)
    for condition, result in conditions:
        # remove `` when there is no % in the column name
        formatted_conditions = []
        for cond in condition.split(' and '):
            if '%' in re.split(r'[<>]', cond)[0]:
                formatted_conditions.append(cond.strip())
            else:
                formatted_conditions.append(cond.replace("`", "").strip())
        output_line = f"{result}," + ",".join(formatted_conditions) + "\n"
        output.append(output_line)
    result=("".join(output))
    return result

query_string = """
select site_key, INITIAL_channel,
case
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' <= 0.23 and 'Top_SKU_case_per_month' <= 0.17 and 'Bottom_SKU_case_per_month' <= 0.38 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' <= 0.23 and 'Top_SKU_case_per_month' > 0.17 and 'Bottom_SKU_case_per_month' <= 0.14 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' <= 0.23 and 'Top_SKU_case_per_month' > 0.17 and 'Bottom_SKU_case_per_month' > 0.14 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' > 0.23 and 'Bottom_SKU_case_per_month' <= 0.12 and 'Top_SKU_trans_month' <= 0.31 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' > 0.23 and 'Bottom_SKU_case_per_month' <= 0.12 and 'Top_SKU_trans_month' > 0.31 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' > 0.23 and 'Bottom_SKU_case_per_month' > 0.12 and 'Sachet_salience' > 0.13 then 'HFS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' <= 0.39 and 'Bottom_SKU_case_per_month' <= 0.11 and 'all_other_size_salience' > 0.28 then 'HFS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' > 0.39 and 'Bottom_SKU_case_per_month' <= 0.02 and 'Top_SKU_trans_month' <= 0.34 then 'HFS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' <= 0.23 and 'Top_SKU_case_per_month' <= 0.17 and 'Bottom_SKU_case_per_month' > 0.38 then 'WS'
	when 'Top_SKU_case_per_month' <= 0.31 and 'Top_SKU_case_per_month' > 0.23 and 'Bottom_SKU_case_per_month' > 0.12 and 'Sachet_salience' <= 0.13 then 'WS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' <= 0.39 and 'Bottom_SKU_case_per_month' <= 0.11 and 'all_other_size_salience' <= 0.28 then 'WS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' <= 0.39 and 'Bottom_SKU_case_per_month' > 0.11 and 'Sachet_salience' <= 0.25 then 'WS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' <= 0.39 and 'Bottom_SKU_case_per_month' > 0.11 and 'Sachet_salience' > 0.25 then 'WS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' > 0.39 and 'Bottom_SKU_case_per_month' <= 0.02 and 'Top_SKU_trans_month' > 0.34 then 'WS'
	when 'Top_SKU_case_per_month' > 0.31 and 'Top_SKU_case_per_month' > 0.39 and 'Bottom_SKU_case_per_month' > 0.02 then 'WS'
	else INITIAL_channel
end as ATLANTIS_auto_channel
from default.vn_eda_features
"""

result=parse_default_stage_rules(query_string)
print(result)

HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' <= 0.23,'Top_SKU_case_per_month' <= 0.17,'Bottom_SKU_case_per_month' <= 0.38
HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' <= 0.23,'Top_SKU_case_per_month' > 0.17,'Bottom_SKU_case_per_month' <= 0.14
HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' <= 0.23,'Top_SKU_case_per_month' > 0.17,'Bottom_SKU_case_per_month' > 0.14
HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' > 0.23,'Bottom_SKU_case_per_month' <= 0.12,'Top_SKU_trans_month' <= 0.31
HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' > 0.23,'Bottom_SKU_case_per_month' <= 0.12,'Top_SKU_trans_month' > 0.31
HFS,'Top_SKU_case_per_month' <= 0.31,'Top_SKU_case_per_month' > 0.23,'Bottom_SKU_case_per_month' > 0.12,'Sachet_salience' > 0.13
HFS,'Top_SKU_case_per_month' > 0.31,'Top_SKU_case_per_month' <= 0.39,'Bottom_SKU_case_per_month' <= 0.11,'all_other_size_salience' > 0.28
HFS,'Top_SKU_case_per_month' > 0.31,'Top_SKU_case_

In [32]:
import pandas as pd
df_auto_separation_concat=pd.DataFrame()

df_auto_separation_concat["Manual1"] = pd.NA
df_auto_separation_concat["Manual1"] = df_auto_separation_concat["Manual1"].astype("string")
df_auto_separation_concat["Manual2"] = pd.NA
df_auto_separation_concat["Manual2"] = df_auto_separation_concat["Manual2"].astype(
"string"
 )
df_auto_separation_concat["Manual3"] = pd.NA
df_auto_separation_concat["Manual3"] = df_auto_separation_concat["Manual3"].astype("string")

print(df_auto_separation_concat["Manual1"].isna().all())

True


In [5]:
import pandas as pd
import dash
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div([html.Img(src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADIAAAAmCAYAAACGeMg8AAAC8klEQVR4nO2Yy24URxSGv6ruqu72LXhQhJERSEgsSPIAEYFtFomyifIKiZQFj8CLZJsFCm+QsMGwhAWKV7ByboBIbHDHTndVT1Wx8IxxPB4z3e6GEZp/N93V55zv/FV1pBEhhMB7IPmuC2hLM5Bp0wxk2jQDmTbFXQb/b+sZ/26sE5AIPMnpVU5d+KiTXJ2ClL//yrOfbvDSCuZkn5UrX3cGIrqc7Cbf5OUfj9BJRmVLsuUzLJ692EmuzkBCCJSlIc1SxOBZYSwqksRx+xuhk8MeQqAoCrRW+xAAqVZYa3HOtZ6zdUeccxhjSJKEKIpG3g8hlVIopVrL26ojIQR2dnbGQgAIIciyjKIosNa2lrs1R4adTtMUKd/cn+H646DrqBVHDhY1CQS8dsYY08qZacWRwjp0JIii+n0JIWDMnpMn6Wuje9Du5vz58w9UpiR4R7LwAee/+B6ipHYsIQR/b1Q8+OU3lI5J5zRXv1klnatXWiOQUJW4x2tsbW4RS4FaWcW7b4lUfRCA/B/Hxv0KG7bp9Zb59CsHNUEaba3gPWZ3G2sMAEmaoeeXEEK84cuj1a8c+YtdhJREUrK4nNWO1fiMGGtRShOCxzmPVs2ntXMO5/tolVCWBWma1Y7R6HR57/HOIQVEUtKvLCe5M6y1xNHecJQyoqqq2jEagVhr0Vrv/1ZKNR5uzjmEEPvXtlLq7YB47wkh/G+IKaVwzjVy5XBThBDEcVwbpjbI4cRDNXHlsBsHY3UKcpQbB5O7wftJNa4pTVypBWJdQB9zoyidUDk/UaxxbgyltcLTn7i2ie7MzYe3eXL3JlXfceaTz1j9/LvRwvp9nt/5ka3nT8l0RO/ja/QuXxlZV5aGfDsnz3MWl/ZmT5KMDtL1tRfcu/UXMgqcu7TMl9fPIRg/WyYCESpFL55GIZDJwphFIJJ5sqUesQyIeHTLDJYhpCBN08EWPbo4pQWnPswIONIFeQzCIO7s3/gp0wxk2jQDmTbNQKZNrwC8MVLggorrHgAAAABJRU5ErkJggg==")
])
if __name__ == '__main__':
    app.run_server(debug=True)

In [4]:
#装饰器
def wrapper_decorator(func):
    def wrapper(*args,**kargs):
        print("wrapper function")
        result=func(*args,**kargs)
        print("wrapperred")
        return result
    return wrapper

@wrapper_decorator
def say_hello(name):
    print(name + " Hello")

say_hello("Jim")

wrapper function
Jim Hello
wrapperred


In [2]:
test=[x*(x+2) for x in range(100)]
print(test)

[0, 3, 8, 15, 24, 35, 48, 63, 80, 99, 120, 143, 168, 195, 224, 255, 288, 323, 360, 399, 440, 483, 528, 575, 624, 675, 728, 783, 840, 899, 960, 1023, 1088, 1155, 1224, 1295, 1368, 1443, 1520, 1599, 1680, 1763, 1848, 1935, 2024, 2115, 2208, 2303, 2400, 2499, 2600, 2703, 2808, 2915, 3024, 3135, 3248, 3363, 3480, 3599, 3720, 3843, 3968, 4095, 4224, 4355, 4488, 4623, 4760, 4899, 5040, 5183, 5328, 5475, 5624, 5775, 5928, 6083, 6240, 6399, 6560, 6723, 6888, 7055, 7224, 7395, 7568, 7743, 7920, 8099, 8280, 8463, 8648, 8835, 9024, 9215, 9408, 9603, 9800, 9999]


In [4]:
#迭代器+生成器
def hello():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3

g=hello()
for res in g:
    print(res)
    

Step 1
1
Step 2
2
Step 3
3


In [5]:
#上下文管理器
class FileManager:
    def __init__(self, file_name, mode):
        self.file_name = file_name
        self.mode = mode
        self.file = None

    def __enter__(self):
        # 当进入上下文时，打开文件
        self.file = open(self.file_name, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        # 离开上下文时，确保关闭文件
        if self.file:
            self.file.close()
        # 返回 `False` 表示异常仍会被抛出
        return False

# 使用 FileManager 上下文管理器
with FileManager("example.txt", "w") as f:
    f.write("Hello, World!")
# 离开 `with` 块，文件会自动关闭

In [6]:
class Logger:
    def __init__(self, log_file):
        self.log_file = log_file
        self.file = None

    def __enter__(self):
        # 打开日志文件进入上下文
        self.file = open(self.log_file, "a")
        print("Initialized Logger, ready to log messages!")  # 控制台日志
        return self  # 返回 Logger 实例

    def __exit__(self, exc_type, exc_val, exc_tb):
        # 离开上下文时关闭文件
        if self.file:
            self.file.close()
        if exc_type is not None:
            print(f"An error occurred: {exc_val}")  # 处理任何错误
        return False  # 异常不会被 suppress

    # 定义一个写日志的方法
    def write_log(self, message):
        if self.file:
            self.file.write(f"{message}\n")
            print(f"Logged message: {message}")  # 控制台日志
        else:
            raise RuntimeError("Logger not initialized. Please use with statement.")

# 使用 Logger 上下文管理器
with Logger("example.log") as logger:
    logger.write_log("This is the first log message.")  # 调用类方法写日志
    logger.write_log("Another log message.")           # 再次调用类方法

Initialized Logger, ready to log messages!
Logged message: This is the first log message.
Logged message: Another log message.


In [22]:
import decimal
# decimal.getcontext().rounding= decimal.ROUND_HALF_UP
a=1.125
a=decimal.Decimal(str(a))
a = a.quantize(decimal.Decimal("0.01"), decimal.ROUND_HALF_UP)
print(a)

1.13
